In [12]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import time

# Load data

In [2]:
df = pd.read_csv("inaug_addr_cleaned.csv", encoding="latin").dropna()
full_text = df["text"].to_string(index=False).replace("\n", " ")
sentences = df["text"].tolist()
df.head()

,doc index,text,P1,P2,Final,IsSame
0,0,Fellow-Citizens of the Senate and of the House...,0.0,0.0,0.0,True
1,0,"On the one hand, I was summoned by my Country,...",0.0,0.0,0.0,True
2,0,"On the other hand, the magnitude and difficult...",0.0,0.0,0.0,True
3,0,In this conflict of emotions all I dare aver i...,0.0,0.0,0.0,True
4,0,"All I dare hope is that if, in executing this ...",0.0,0.0,0.0,True


# Preprocessing
- tokenize only the top 2500 words
- vectorize the text and convert it into sequence of integers 
- pad_sequences to convert the sequences into 2-D numpy array

In [3]:
tokenizer = Tokenizer(num_words=2500, lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(df['text'].values)
#print(tokenizer.word_index)  # To see the dicstionary
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [4]:
Y = pd.get_dummies(df['Final']).values
Y.shape

(4847, 2)

In [5]:
print("{} rows of data; {} features.".format(X.shape[0], X.shape[1]))


4847 rows of data; 663 features.


# Built LSTM network
- hyperparameters
    - embed_dim = 128
    - lstm_out = 196
    - batch_size = 32
- activation function
    - softmax


In [9]:
# hyperparameters
embed_dim = 128
lstm_out = 196
batch_size = 32

model = Sequential()
model.add(Embedding(2500, embed_dim, input_length = X.shape[1]))
model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 663, 128)          320000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 575,194
Trainable params: 575,194
Non-trainable params: 0
_________________________________________________________________
None


# Training

In [10]:
# train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3877, 663) (3877, 2)
(970, 663) (970, 2)


In [13]:
t0=time.time()
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 1, verbose = 2)
print("Run time: {} s".format(time.time()-t0))

Epoch 1/1
 - 120s - loss: 0.1469 - acc: 0.9574
Run time: 119.52261877059937 s


# Testing

In [15]:
score, acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.4f" % (score))
print("Validation Accuracy: %.4f" % (acc))

Score: 0.1905
Validation Accuracy: 0.9546
